In [1]:
from kalman_gst import *  
from pygsti.modelpacks import smq1Q_XYI as std
import random

In [2]:
SAMPLES = 256

In [3]:
# setup the FOGI model
mdl_datagen = std.target_model('H+s')
basis1q = pygsti.baseobjs.Basis.cast('pp', 4)
gauge_basis = pygsti.baseobjs.CompleteElementaryErrorgenBasis(
                        basis1q, mdl_datagen.state_space, elementary_errorgen_types='HS')
mdl_datagen.setup_fogi(gauge_basis, None, None, reparameterize=True,
                     dependent_fogi_action='drop', include_spam=True)
target_model = mdl_datagen.copy()

In [4]:
# model labels
mdl_datagen.fogi_errorgen_component_labels()

('H(X:0)_[]',
 'H(Y:0)_[]',
 'H(Z:0)_[]',
 'S(X:0)_[]',
 'S(Y:0)_[]',
 'S(Z:0)_[]',
 'H(X:0)_Gxpi2:0',
 'S(X:0)_Gxpi2:0',
 '(0.5 S(Y:0) + 0.5 S(Z:0))_Gxpi2:0',
 'H(Y:0)_Gypi2:0',
 '(0.5 S(X:0) + 0.5 S(Z:0))_Gypi2:0',
 'S(Y:0)_Gypi2:0',
 'ga(-H(Z:0))_Gypi2:0 - ga(-H(Z:0))_Gxpi2:0',
 'ga(H(Y:0))_rho0 - ga(H(Y:0))_Gxpi2:0',
 'ga(-H(Y:0))_Mdefault - ga(-H(Y:0))_Gxpi2:0',
 'ga(H(X:0))_rho0 - ga(H(X:0))_Gypi2:0',
 'ga(-H(X:0))_Mdefault - ga(-H(X:0))_Gypi2:0',
 'ga(-0.5 S(X:0) - 0.5 S(Y:0))_Mdefault - ga(-0.5 S(X:0) - 0.5 S(Y:0))_rho0')

In [5]:
# single out hamiltonian parameters
hamiltonian_params = [0, 1, 2, 6, 9]
print([mdl_datagen.fogi_errorgen_component_labels()[i] for i in hamiltonian_params])

['H(X:0)_[]', 'H(Y:0)_[]', 'H(Z:0)_[]', 'H(X:0)_Gxpi2:0', 'H(Y:0)_Gypi2:0']


In [6]:
# add noise to the stochastic and hamiltonian parts of the FOGI rates
SEED = 2023
np.random.seed(SEED)

max_stochastic_error_rate = 0.001
hamiltonian_error_var = 0.05
ar = mdl_datagen.fogi_errorgen_components_array(include_fogv=False, normalized_elem_gens=True)

for i in range(len(ar)):
    if i in hamiltonian_params:
        ar[i] = np.random.normal(0, hamiltonian_error_var)



# add hamiltonian noise
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)
print('hamiltonian-only MSE with target', mserror(mdl_datagen, target_model))
print('hamiltonian-only agsi with target', avg_gs_infidelity(mdl_datagen, target_model))

for i in range(len(ar)):
    if i not in hamiltonian_params:
        ar[i] = max_stochastic_error_rate*np.random.rand(1)

# add stochastic noise
mdl_datagen.set_fogi_errorgen_components_array(ar, include_fogv=False, normalized_elem_gens=True)

print('MSE with target', mserror(mdl_datagen, target_model))
print('agi with target', avg_gs_infidelity(mdl_datagen, target_model))

hamiltonian-only MSE with target 0.0042044138696715845
hamiltonian-only agsi with target 0.0008397945630547853
MSE with target 0.004206609448553395
agi with target 0.001550380292738862


In [7]:
print('model is cptp', model_is_cptp(mdl_datagen))

model is cptp True


In [8]:
# make a GST edesign and simulate the data
maxLengths = [1,2,4,8,16,32,64,128,256,512]
edesign = pygsti.protocols.StandardGSTDesign(target_model, std.prep_fiducials(), std.meas_fiducials(),
                                                std.germs(), maxLengths)
dataset = pygsti.data.simulate_data(mdl_datagen, edesign, SAMPLES, seed=SEED) #, sample_error='none')

In [9]:
# make MLE estimates
mle_estimates, edesigns = make_mle_estimates(dataset, std, target_model, maxLengths)

--- Iterative GST: [##################################################] 100.0%  92 circuits ---
Iterative GST Total Time: 1.2s
--- Iterative GST: [##################################################] 100.0%  168 circuits ---
Iterative GST Total Time: 1.0s
--- Iterative GST: [##################################################] 100.0%  285 circuits ---
Iterative GST Total Time: 2.8s
--- Iterative GST: [##################################################] 100.0%  448 circuits ---
Iterative GST Total Time: 3.6s
--- Iterative GST: [##################################################] 100.0%  616 circuits ---
Iterative GST Total Time: 5.0s
--- Iterative GST: [##################################################] 100.0%  784 circuits ---
Iterative GST Total Time: 5.5s
--- Iterative GST: [##################################################] 100.0%  952 circuits ---
Iterative GST Total Time: 7.3s
--- Iterative GST: [##################################################] 100.0%  1120 circuits ---
Iterati

In [10]:
# set germ circuit ranges for the mle batches
germ_length_ranges = {
    0: [0, 92], 
    1: [92, 168], 
    2: [168, 285], 
    3: [285, 448],
    4: [448, 616],
    5: [616, 784], 
    6: [784, 952], 
    7: [952, 1120], 
    8: [1120, 1288],
    9: [1288, 1456],
}

### Cells below consitute a "standard practice" Kalman estimation routine

In [ ]:
# run a direct RB experiment
from pygsti.processors import CliffordCompilationRules as CCR
compilations = {'absolute': CCR.create_standard(std.processor_spec(), 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(std.processor_spec(), 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}
depths = [0,1,2,4,8,16,32,64,128,256,512]
k = 10
# To run direct / mirror RB change CliffordRBDesign -> DirectRBDesign / MirrorRBDesign
rb_edesign = pygsti.protocols.CliffordRBDesign(std.processor_spec(), compilations, depths, k)
rb_dataset = pygsti.data.simulate_data(mdl_datagen, rb_edesign, 1000)
rb_data = pygsti.protocols.ProtocolData(rb_edesign, rb_dataset)
rb_protocol = pygsti.protocols.RB()
rb_results = rb_protocol.run(rb_data)
rb_rate = rb_results.fits['full'].estimates['r']

- Sampling 10 circuits at CRB length 0 (1 of 11 depths) with seed 15794
- Sampling 10 circuits at CRB length 1 (2 of 11 depths) with seed 15804
- Sampling 10 circuits at CRB length 2 (3 of 11 depths) with seed 15814
- Sampling 10 circuits at CRB length 4 (4 of 11 depths) with seed 15824
- Sampling 10 circuits at CRB length 8 (5 of 11 depths) with seed 15834
- Sampling 10 circuits at CRB length 16 (6 of 11 depths) with seed 15844
- Sampling 10 circuits at CRB length 32 (7 of 11 depths) with seed 15854
- Sampling 10 circuits at CRB length 64 (8 of 11 depths) with seed 15864
- Sampling 10 circuits at CRB length 128 (9 of 11 depths) with seed 15874
- Sampling 10 circuits at CRB length 256 (10 of 11 depths) with seed 15884
- Sampling 10 circuits at CRB length 512 (11 of 11 depths) with seed 15894


In [ ]:
# randomize the germ batches 
random.seed(SEED)
circuits = [c for c in edesign.circuit_lists[-1]]
for i in range(-1, len(maxLengths)-1, 1):
    if i == -1:
        subcircs = circuits[0:len(edesign.circuit_lists[i+1])]
    else:
        subcircs = circuits[len(edesign.circuit_lists[i]):len(edesign.circuit_lists[i+1])]
    np.random.shuffle(subcircs)
    if i == -1:
        circuits[0:len(edesign.circuit_lists[i+1])] = subcircs
    else:
        circuits[len(edesign.circuit_lists[i]):len(edesign.circuit_lists[i+1])] = subcircs    

In [ ]:
# filter the dataset
prior_covar_strength =  rb_rate/target_model.num_params
prior_covar = prior_covar_strength*np.eye(target_model.num_params)

ekf = ExtendedKalmanFilter(target_model, prior_covar)

ekf.filter_dataset(circuits, dataset)

In [ ]:
# MSE evolution plots
fig = plt.figure(figsize=(6, 3))
ax = fig.add_subplot()
ax.set_title('1 qubit mean square error', fontsize=14)

true_params = mdl_datagen.to_vector()
ekf_error = []
ekf_uncert = []

for i in range(len(ekf.param_history)):
    ekf_error.append( ((ekf.param_history[i]-true_params)@(ekf.param_history[i]-true_params))/ekf.model.num_params) 
    ekf_uncert.append( (np.trace(ekf.covar_history[i]))/ekf.model.num_params) 


ekf_line, = ax.plot(ekf_error, c='blue', linestyle='solid')
ekf_uline, = ax.plot(ekf_uncert, c='blue', linestyle='dotted')

if mle_estimates is not None:
    for i, mdl in enumerate(mle_estimates):
        mle_error = mserror(mdl, mdl_datagen)/ekf.model.num_params
        mle_line, = ax.semilogy(germ_length_ranges[i], (mle_error, mle_error), c='gray', label='MLE Estimate')
        
plt.legend(['Point estimate error', 'Expected error', 'batched MLE error'])


ax.set_xlabel('GST Circuit Index', fontsize=14)
ax.set_ylabel('MSE', fontsize=14)
plt.savefig('Figures/1qMSE.eps', dpi=350, format="eps", bbox_inches='tight')

In [ ]:
# MAE (mean absolute error) evolution plots
fig = plt.figure(figsize=(6, 3))
ax = fig.add_subplot()
ax.set_title('1-qubit mean absolute error', fontsize=14)

datagen_vec = mdl_datagen.to_vector()
estimates = [e.to_vector() for e in mle_estimates]
ekf_error = []
ekf_uncert = []

param_hist = ekf.param_history
covar_hist = ekf.covar_history

for i in range(len(param_hist)):
    ekf_error.append( sum(abs((param_hist[i]-datagen_vec))) /len(param_hist[i]) ) 
    ekf_uncert.append( (np.trace(np.sqrt(covar_hist[i]))/len(param_hist[i]))  ) 


ekf_uline, = ax.semilogy(ekf_uncert, c='blue', linestyle='dotted')
ekf_line, = ax.plot(ekf_error, c='blue', linestyle='solid')

for i, evec in enumerate(estimates):
    mle_error = sum(abs(evec - datagen_vec))/len(evec)
    mle_line, = ax.plot(germ_length_ranges[i], (mle_error, mle_error), c='gray', label='MLE Estimate')
ax.set_xlabel('GST Circuit Index', fontsize=14)
ax.set_ylabel('MAE', fontsize=14)
plt.savefig('Figures/1qMAE.eps', dpi=350, format="eps", bbox_inches='tight')

In [ ]:
# Hamiltonian parameter error plots


ekf_mserrors = np.zeros((len(hamiltonian_params), len(ekf.param_history)))
ekf_var = np.zeros((len(hamiltonian_params), len(ekf.param_history)))
true_params = mdl_datagen.to_vector()

for i in range(len(ekf.param_history)):
    for j in range(len(hamiltonian_params)):
        ekf_mserrors[j, i] = ekf.param_history[i][hamiltonian_params[j]]-true_params[hamiltonian_params[j]]
        ekf_var[j, i] = ekf.covar_history[i][hamiltonian_params[j],hamiltonian_params[j]]
param_lines = []   
for j in range(len(hamiltonian_params)):
    line, = plt.plot(ekf_mserrors[j, :])
    sigma = np.sqrt(ekf_var[j, :])
    param_lines.append(line)
    plt.fill_between(range(len(ekf_mserrors[j, :])), ekf_mserrors[j, :]-sigma, ekf_mserrors[j, :]+sigma, alpha=0.5)
plt.legend(param_lines, [mdl_datagen.fogi_errorgen_component_labels()[i] for i in hamiltonian_params])

plt.title('Evolution of FOGI Hamiltonian error')
plt.xlabel('Circuit index')
plt.ylabel('Estimate error')

In [ ]:
# Over-rotation error plot
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot()
ax.set_title('1 qubit over-rotation estimate', fontsize=14)

overrot_errors = np.zeros((2, len(ekf.param_history)))
true_params = mdl_datagen.to_vector()

for i in range(len(ekf.param_history)):
    overrot_errors[0, i] = ekf.param_history[i][6]
    overrot_errors[1, i] = ekf.param_history[i][9]

xline_estimate, = ax.semilogx(overrot_errors[0, :], c='#d53e4f')
sigmas = np.sqrt([c[6,6] for c in ekf.covar_history])
ax.fill_between(range(len(overrot_errors[0, :])), overrot_errors[0, :]-sigmas, overrot_errors[0, :]+sigmas, alpha=0.5, color='#d53e4f', linewidth=0.0)
yline_estimate, = ax.semilogx(overrot_errors[1, :], c='#3288bd')
sigmas = np.sqrt([c[9,9] for c in ekf.covar_history])
ax.fill_between(range(len(overrot_errors[1, :])), overrot_errors[1, :]-sigmas, overrot_errors[1, :]+sigmas, alpha=0.5, color='#3288bd', linewidth=0.0)

xline_true, = ax.plot((0, len(overrot_errors[0, :])), (true_params[6], true_params[6]), c='#d53e4f', linestyle='dotted')
yline_true, = ax.plot((0, len(overrot_errors[0, :])), (true_params[9], true_params[9]), c='#3288bd', linestyle='dotted')


ax.legend([xline_estimate, yline_estimate], ['X gate error', 'Y gate error', 'True X gate error', 'True Y gate error'], loc='upper right', title='Estimates')

ax.set_xlabel('GST Circuit index', fontsize=14)
ax.set_ylabel('Estimate (radians)', fontsize=14)
plt.savefig('Figures/1qOverrot.png', dpi=350, format="png", bbox_inches='tight')